In [11]:
# @title Upload CSV
import random
import copy
import numpy as np
import pandas as pd
from datetime import datetime
from collections import defaultdict

# Fungsi Bantu
def time_to_minutes(t):
    try:
        dt = datetime.strptime(t, "%H:%M:%S")
    except ValueError:
        dt = datetime.strptime(t, "%H:%M")
    return dt.hour * 60 + dt.minute

# Inisialisasi Data
dosen_df = pd.read_csv('data_skripsi_dosen.csv')
mk_genap_df = pd.read_csv('data_skripsi_mk_genap.csv')
data_dosen_df = pd.read_csv('data_skripsi_data_dosen.csv')
hari_df = pd.read_csv('data_skripsi_hari.csv')
ruang_df = pd.read_csv('data_skripsi_ruang.csv')
jam_df = pd.read_csv('data_skripsi_jam.csv')

# Urutkan jam_df sebelum generate slot
jam_df = jam_df.sort_values('id_jam')

merged_df = pd.merge(
    pd.merge(data_dosen_df, dosen_df, on='id_dosen'),
    mk_genap_df, on='id_mk_genap'
)

print("Data Gabungan:")
print(merged_df.head())

total_sks = merged_df['sks'].sum()
print(f"Total slot SKS yang dibutuhkan: {total_sks}")
print(f"Total data dalam merged_df: {len(merged_df)}")

Data Gabungan:
   id_dosen  id_mk_genap kelas                   nama_dosen  \
0         2    211840231     A  Ahmad Azhari, S.Kom., M.Eng   
1         2    211861331     A  Ahmad Azhari, S.Kom., M.Eng   
2         2    211861431     A  Ahmad Azhari, S.Kom., M.Eng   
3         3    211840131     A     Ali Tarmuji, S.T., M.Cs.   
4         3    211860330     A     Ali Tarmuji, S.T., M.Cs.   

                              nama_mk_genap  smt  sks    sifat kategori  \
0                          Grafika Komputer    4    3    Wajib        -   
1                Pengembangan Aplikasi Game    6    3  Pilihan       SC   
2                          Pengenalan Pola     6    3  Pilihan       SC   
3  Analisis dan Perancangan Perangkat Lunak    4    3    Wajib        -   
4                  Rekayasa Perangkat Lunak    6    3    Wajib        -   

    metode  
0  Offline  
1  Offline  
2   Online  
3  Offline  
4  Offline  
Total slot SKS yang dibutuhkan: 282
Total data dalam merged_df: 107


Tahapan GWO

1. Preprocessing
    
    Membangun struktur slot waktu.

2. Inisialisasi Populasi (GWO)
    

    Inisialisasi populasi serigala acak (Xi).

    Setiap "serigala" dalam GWO mewakili solusi penjadwalan yang mungkin (misalnya: variasi pengaturan slot).

3. Fitness Function (GWO)
    Hitung nilai fitness untuk setiap serigala.

    Mengevaluasi kualitas penjadwalan (misalnya: minimalisasi konflik, kepadatan ruang, dll).

4. Proses Optimasi (GWO)
    Tentukan Alpha, Beta, dan Delta berdasarkan nilai fitness.

    Menggunakan hierarki Alpha, Beta, Delta untuk memperbarui posisi solusi.

    While (iterasi < maksimum iterasi):
      1. Perbarui parameter a, A, dan C.
      2. For setiap serigala:
          1. Perbarui posisi serigala berdasarkan Alpha, Beta, dan Delta.
      3. Hitung nilai fitness untuk setiap serigala.
      4. Perbarui Alpha, Beta, dan Delta.
5. Postprocessing
    Return solusi terbaik (Alpha).
    
    Menampilkan jadwal terbaik hasil optimasi.


In [12]:
# @title Preprosessing
hari_list = hari_df['nama_hari'].tolist()
ruang_list = ruang_df['nama_ruang'].tolist()
jam_list = jam_df[['id_jam', 'jam_awal', 'jam_akhir']].to_dict('records')
mata_kuliah_list = mk_genap_df.set_index('id_mk_genap').to_dict('index')

def slot_generator():
    wolf = []
    id_counter = 1
    for hari in hari_df['nama_hari']:
        for ruang in ruang_df['nama_ruang']:
            for jam in jam_df.itertuples():
                wolf.append({
                    "id_slot": id_counter,
                    "mata_kuliah": None,
                    "dosen": None,
                    "ruang": ruang,
                    "hari": hari,
                    "jam_mulai": jam.jam_awal,
                    "jam_selesai": jam.jam_akhir,
                    "kelas": None,
                    "sks": None,
                    "metode": None
                })
                id_counter += 1
    return wolf

slots = slot_generator()
for slot in slots[:5]:
    print(slot)

print("\n")
for slot in slots[-5:]:
    print(slot)

{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 5, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': None, 'sks': None, 'metode': None}


{'id_slot': 500, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.63', 'hari'

In [13]:
# Fungsi utama: buat jadwal acak
def create_random_schedule(ordered_courses_df=None):
    schedule = slot_generator()
    
    # Gunakan parameter jika disediakan, acak jika tidak
    if ordered_courses_df is None:
        merged_shuffled = merged_df.sample(frac=1).iterrows()
    else:
        merged_shuffled = ordered_courses_df.iterrows()

    # Tracking alokasi, meskipun pengecekan overlap tidak digunakan, tetap disimpan sebagai referensi
    room_allocations = defaultdict(list)    # key: (ruang, hari) -> list of (start, end)
    teacher_allocations = defaultdict(list)  # key: (dosen, hari) -> list of (start, end)
    class_allocations = defaultdict(list)    # key: (kelas, hari) -> list of (start, end)

    for _, row in merged_shuffled:
        mata_kuliah = row['nama_mk_genap']
        dosen = row['nama_dosen']
        kelas = row['kelas']
        sks = int(row['sks'])
        metode = row['metode']

        possible_positions = list(range(len(schedule) - sks + 1))
        random.shuffle(possible_positions)
        candidate_blocks = []

        for i in possible_positions:
            block = schedule[i:i+sks]
            # Pastikan semua slot kosong dan berada di hari yang sama
            if not all(slot['mata_kuliah'] is None for slot in block) or not all(slot['hari'] == block[0]['hari'] for slot in block):
                continue
            # Untuk offline, cek kesamaan ruangan
            if metode != 'Online' and not all(slot['ruang'] == block[0]['ruang'] for slot in block):
                continue
            # Pengecekan waktu berurutan dihapus, sehingga blok diperbolehkan memiliki gap waktu

            hari = block[0]['hari']
            ruang = "Online" if metode == 'Online' else block[0]['ruang']
            time_block = (time_to_minutes(block[0]['jam_mulai']), time_to_minutes(block[-1]['jam_selesai']))

            # Pengecekan overlap dihapus, jadi langsung tambahkan blok kandidat
            kelas_already = len(class_allocations[(kelas, hari)]) > 0
            candidate_blocks.append((block, time_block, kelas_already))

        # Pemilihan blok kandidat: ambil blok pertama jika ada kandidat
        selected_block = candidate_blocks[0][0] if candidate_blocks else None

        if selected_block is None:
            print(f"Gagal menempatkan: {kelas} - {mata_kuliah} - {dosen}")
        else:
            for slot in selected_block:
                slot.update({
                    "mata_kuliah": mata_kuliah,
                    "dosen": dosen,
                    "kelas": kelas,
                    "sks": sks,
                    "metode": metode,
                    "ruang": "Online" if metode == 'Online' else slot['ruang']
                })
            hari = selected_block[0]['hari']
            ruang = "Online" if metode == 'Online' else selected_block[0]['ruang']
            time_block = (time_to_minutes(selected_block[0]['jam_mulai']),
                          time_to_minutes(selected_block[-1]['jam_selesai']))

            if metode != 'Online':
                room_allocations[(ruang, hari)].append(time_block)
            teacher_allocations[(dosen, hari)].append(time_block)
            class_allocations[(kelas, hari)].append(time_block)

    return schedule

# Contoh penggunaan:
population_size = 1
population = [create_random_schedule() for _ in range(population_size)]

for schedule in population:
    for slot in schedule:
        print(slot)

{'id_slot': 1, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:00:00', 'jam_selesai': '7:50:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 2, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '7:50:00', 'jam_selesai': '8:45:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 3, 'mata_kuliah': None, 'dosen': None, 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '8:45:00', 'jam_selesai': '9:35:00', 'kelas': None, 'sks': None, 'metode': None}
{'id_slot': 4, 'mata_kuliah': 'Pemrograman Web', 'dosen': 'Ali Tarmuji, S.T., M.Cs.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '9:35:00', 'jam_selesai': '10:25:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot': 5, 'mata_kuliah': 'Pemrograman Web', 'dosen': 'Ali Tarmuji, S.T., M.Cs.', 'ruang': '4.1.5.55', 'hari': 'Senin', 'jam_mulai': '10:30:00', 'jam_selesai': '11:20:00', 'kelas': 'A', 'sks': 3, 'metode': 'Offline'}
{'id_slot'

Konstrain berat 1
Konstrain ringan 0.5

Konstrain berat:
dosen tidak boleh mengajar mata kuliah/kelas berbeda pada jam yang sama
ruang kelas tidak boleh digunakan lebih dari 1 kelas/dosen
3 sks membutuhkan 3 slot waktu
ruangan antar kelas harus sama
slot waktu harus berurutan

Konstrain ringan:
kelas dijadwalkan pada waktu paling akhir dan paling awal pada hari itu

In [14]:
# def calculate_fitness(schedule):
#     penalty = 0

#     teacher_conflicts = []              # Hard constraint: dosen mengajar dua kelas bersamaan
#     room_conflicts = []                 # Hard constraint: ruangan dipakai lebih dari satu kelas bersamaan
    
#     teacher_preference_conflicts = []   # Soft constraint: preferensi dosen
#     day_conflicts = []             # Soft constraint: hari Sabtu

#     # Tracking hard constraint
#     teacher_intervals = defaultdict(list)
#     room_intervals = defaultdict(list)

#     for slot in schedule:
#         if not slot['mata_kuliah']:
#             continue

#         day = slot['hari']
#         teacher = slot['dosen']
#         room = slot['ruang']
#         kelas = slot['kelas']
#         id_slot = slot['id_slot']
#         start = time_to_minutes(slot['jam_mulai'])
#         end = time_to_minutes(slot['jam_selesai'])
        
#         teacher_intervals[(teacher, day)].append((start, end, id_slot))
#         if room != "Online":
#             room_intervals[(room, day)].append((start, end, id_slot))
        
#         # Soft constraint: preferensi dosen
#         if str(teacher) in ["Ardiansyah, Dr., S.T., M.Cs."]:
#             if start <= 720:
#                 teacher_preference_conflicts.append({
#                     'teacher': teacher,
#                     'id_slot': id_slot,
#                     'jam': slot['jam_mulai'],
#                     'message': "Hanya ingin dijadwalkan pada jam 12 atau lebih"
#                 })
#                 penalty += 0.5

#         # Soft constraint: setiap slot pada hari Sabtu
#         if str(teacher) in ["Tedy Setiadi, Drs., M.T."]:
#             if day == "Sabtu":
#                 day_conflicts.append({
#                     'id_slot': id_slot,
#                     'teacher': teacher,
#                     'kelas': kelas,
#                     'hari': day,
#                     'message': "Kelas tidak ingin dijadwalkan pada hari Sabtu"
#                 })
#                 penalty += 0.5

#     # Hard constraint: Dosen
#     for (teacher, day), intervals in teacher_intervals.items():
#         intervals.sort(key=lambda x: x[0])
#         for i in range(1, len(intervals)):
#             prev = intervals[i-1]
#             curr = intervals[i]
#             if curr[0] < prev[1]:
#                 teacher_conflicts.append({
#                     'teacher': teacher,
#                     'conflict_ids': (prev[2], curr[2])
#                 })
#                 penalty += 1

#     # Hard constraint: Ruangan
#     for (room, day), intervals in room_intervals.items():
#         intervals.sort(key=lambda x: x[0])
#         for i in range(1, len(intervals)):
#             prev = intervals[i-1]
#             curr = intervals[i]
#             if curr[0] < prev[1]:
#                 room_conflicts.append({
#                     'room': room,
#                     'day': day,
#                     'conflict_ids': (prev[2], curr[2]),
#                     'messages': "dipakai lebih dari 1 kelas"
#                 })
#                 penalty += 1
    
#     return {
#         'penalty': penalty,
#         'violations': {
#             'hard_teacher': len(teacher_conflicts),
#             'hard_room': len(room_conflicts),
#             'teacher_preference': len(teacher_preference_conflicts),
#             'day_conflicts': len(day_conflicts)
#         },
#         'teacher_conflicts': teacher_conflicts,
#         'room_conflicts': room_conflicts,
#         'teacher_preference_conflicts': teacher_preference_conflicts,
#         'day_conflicts': day_conflicts
#     }

# # validasi ruangan
# # def compare_room_usage(schedule):
# #     room_usage = defaultdict(list)
    
# #     for slot in schedule:
# #         if not slot['mata_kuliah']:
# #             continue
# #         room = slot['ruang']
# #         usage_info = {
# #             'id_slot': slot['id_slot'],
# #             'dosen': slot['dosen'],
# #             'kelas': slot['kelas'],
# #             'hari': slot['hari'],
# #             'jam_mulai': slot['jam_mulai'],
# #             'jam_selesai': slot['jam_selesai']
# #         }
# #         room_usage[room].append(usage_info)
    
# #     for room in room_usage:
# #         room_usage[room].sort(key=lambda x: (x['hari'], time_to_minutes(x['jam_mulai'])))
# #     return room_usage

# population_size = 1
# population = [create_random_schedule() for _ in range(population_size)]

# for idx, schedule in enumerate(population):
#     fitness = calculate_fitness(schedule)
#     print(f"\nIndividu {idx+1}:")
#     for slot in schedule:
#         print(slot)

#     print(f"\nTotal Penalty: {fitness['penalty']}")
    
#     # validasi konflik dosen (hard constraint)
#     print("\nDetail Konflik Dosen:")
#     if not fitness['teacher_conflicts']:
#         print(" - Tidak ada konflik dosen.")
#     else:
#         for conflict in fitness['teacher_conflicts']:
#             conflict_ids = conflict['conflict_ids']
#             print(f" - {conflict['teacher']}: Konflik pada Slot {conflict_ids[0]} dan {conflict_ids[1]}")
    
#     # validasi ruangan (hard constraint)
#     print("\nValidasi Data Ruangan:")
#     room_conflicts = fitness['room_conflicts']
#     if not room_conflicts:
#         print(" - Data ruangan valid, tidak ada konflik.")
#     else:
#         for conflict in room_conflicts:
#             conflict_ids = conflict['conflict_ids']
#             print(f" - Ruang {conflict['room']} {conflict['message']} : Konflik pada Slot {conflict_ids[0]} dan {conflict_ids[1]}")

#     # Soft constraint: preferensi dosen
#     print("\nSoft Constraint - Preferensi Dosen (Jam 6 sampai 12):")
#     if not fitness['teacher_preference_conflicts']:
#         print(" - Tidak ada konflik preferensi dosen.")
#     else:
#         for conflict in fitness['teacher_preference_conflicts']:
#             print(f" - Slot {conflict['id_slot']} - {conflict['teacher']} (jam mulai: {conflict['jam']}): {conflict['message']}")

#     # Soft constraint: hari Sabtu
#     print("\nSoft Constraint - Preferensi Dosen selain hari sabtu:")
#     if not fitness['day_conflicts']:
#         print(f" - Data valid, tidak ada konflik.")
#     else:
#         for conflict in fitness['day_conflicts']:
#             print(f" - Slot {conflict['id_slot']} - {conflict['teacher']}, Kelas {conflict['kelas']}: {conflict['message']}")

#     # Komparasi Penggunaan Ruangan Berdasarkan Ruangan
#     # print("\nKomparasi Penggunaan Ruangan Berdasarkan Ruangan:")
#     # room_usage = compare_room_usage(schedule)
#     # if not room_usage:
#     #     print(" - Tidak ada data penggunaan ruangan.")
#     # else:
#     #     for room, usages in room_usage.items():
#     #         print(f" - Ruang: {room}")
#     #         for usage in usages:
#     #             print(f"    ID Slot: {usage['id_slot']}, Dosen: {usage['dosen']}, Kelas: {usage['kelas']}, Hari: {usage['hari']}, Jam Mulai: {usage['jam_mulai']}, Jam Selesai: {usage['jam_selesai']}")


In [15]:
# Fungsi Fitness dengan Perbaikan
def calculate_fitness(schedule):
    penalty = 0
    teacher_conflicts = []
    room_conflicts = []
    teacher_preference_conflicts = []
    saturday_conflicts = []

    teacher_intervals = defaultdict(list)
    room_intervals = defaultdict(list)

    for slot in schedule:
        if not slot['mata_kuliah']:
            continue

        # Konversi waktu
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        
        # Hard constraint: Konflik dosen dan ruangan
        teacher_intervals[(slot['dosen'], slot['hari'])].append((start, end, slot['id_slot']))
        if slot['metode'] != 'Online':
            room_intervals[(slot['ruang'], slot['hari'])].append((start, end, slot['id_slot']))
        
        # Soft constraint: Preferensi dosen
        if "Ardiansyah, Dr., S.T., M.Cs." in str(slot['dosen']) and start <= 720:
            teacher_preference_conflicts.append(slot['id_slot'])
            penalty += 0.5
        
        # Soft constraint: Hari Sabtu
        if "Ali Tarmuji, S.T., M.Cs." in str(slot['dosen']) and slot['hari'] == 'sabtu':
            saturday_conflicts.append(slot['id_slot'])
            penalty += 0.5

    # Hitung konflik dosen
    for key, intervals in teacher_intervals.items():
        intervals.sort()
        for i in range(1, len(intervals)):
            if intervals[i][0] < intervals[i-1][1]:
                penalty += 1
                teacher_conflicts.append({
                    'dosen': key[0],
                    'hari': key[1],
                    'slot1': intervals[i-1][2],
                    'slot2': intervals[i][2]
                })

    # Hitung konflik ruangan
    for key, intervals in room_intervals.items():
        intervals.sort()
        for i in range(1, len(intervals)):
            if intervals[i][0] < intervals[i-1][1]:
                penalty += 1
                room_conflicts.append({
                    'ruang': key[0],
                    'hari': key[1],
                    'slot1': intervals[i-1][2],
                    'slot2': intervals[i][2]
                })

    return penalty

population_size = 5
population = [create_random_schedule() for _ in range(population_size)]

for idx, schedule in enumerate(population):
    fitness = calculate_fitness(schedule)
    print(f"\nIndividu {idx+1}:")
    print(f"Total Penalty: {fitness}")


Individu 1:
Total Penalty: 18.5

Individu 2:
Total Penalty: 8.5

Individu 3:
Total Penalty: 17.0

Individu 4:
Total Penalty: 12.0

Individu 5:
Total Penalty: 20.0


In [18]:
import numpy as np
import random
import copy
from collections import defaultdict

def grey_wolf_optimizer(merged_df, max_iterations=50, num_wolves=10, a_start=2.0, a_end=0.0):
    """
    Implementasi Grey Wolf Optimizer untuk penjadwalan
    
    Parameters:
    merged_df: DataFrame berisi mata kuliah yang akan dijadwalkan
    max_iterations: Jumlah maksimum iterasi
    num_wolves: Jumlah serigala (solusi) dalam populasi
    a_start: Nilai awal parameter a
    a_end: Nilai akhir parameter a
    
    Returns:
    best_schedule: Jadwal terbaik yang ditemukan
    """
    # Inisialisasi populasi (wolves)
    wolves = []
    for _ in range(num_wolves):
        # Acak urutan mata kuliah untuk setiap serigala
        wolf_df = merged_df.sample(frac=1)
        # Buat jadwal berdasarkan urutan mata kuliah
        wolf_schedule = create_random_schedule(wolf_df)
        wolf_fitness = calculate_fitness(wolf_schedule)
        wolves.append({
            'schedule': wolf_schedule,
            'fitness': wolf_fitness
        })
    
    # Inisialisasi alfa, beta, dan delta
    wolves.sort(key=lambda x: x['fitness'])
    alpha = wolves[0]
    beta = wolves[1]
    delta = wolves[2]
    
    # History untuk tracking kemajuan
    history = []
    
    # Iterasi utama
    for iteration in range(max_iterations):
        # Update parameter a
        a = a_start - iteration * ((a_start - a_end) / max_iterations)
        
        # Update posisi setiap serigala (kecuali alpha, beta, delta)
        for i in range(num_wolves):
            # Simpan jadwal asal
            original_schedule = copy.deepcopy(wolves[i]['schedule'])
            
            # Jangan update alpha, beta, delta secara langsung
            if i <= 2:
                continue
            
            # Dapatkan slot dengan konflik
            conflict_slots = get_conflict_slots(original_schedule)
            
            # Jika tidak ada slot konflik, lakukan mutasi acak untuk eksplorasi
            if not conflict_slots and random.random() < 0.3:
                mutated_schedule = mutate_schedule(original_schedule)
                wolves[i]['schedule'] = mutated_schedule
                wolves[i]['fitness'] = calculate_fitness(mutated_schedule)
                continue
            
            # Untuk setiap slot konflik, update posisi berdasarkan alpha, beta, delta
            improved = False
            for slot_id in conflict_slots:
                # Hitung komponen A dan C untuk alpha, beta, delta
                A1, A2, A3 = [2 * a * random.random() - a for _ in range(3)]
                C1, C2, C3 = [2 * random.random() for _ in range(3)]
                
                # Komponen D untuk alpha, beta, delta
                D_alpha = abs(C1 * find_slot_position(alpha['schedule'], slot_id) - find_slot_position(original_schedule, slot_id))
                D_beta = abs(C2 * find_slot_position(beta['schedule'], slot_id) - find_slot_position(original_schedule, slot_id))
                D_delta = abs(C3 * find_slot_position(delta['schedule'], slot_id) - find_slot_position(original_schedule, slot_id))
                
                # Komponen X untuk alpha, beta, delta (mencoba berbagai posisi slot)
                X1 = find_slot_position(alpha['schedule'], slot_id) - A1 * D_alpha
                X2 = find_slot_position(beta['schedule'], slot_id) - A2 * D_beta
                X3 = find_slot_position(delta['schedule'], slot_id) - A3 * D_delta
                
                # Rata-rata menuju posisi baru (ini akan dikonversi ke slot baru)
                new_position = int((X1 + X2 + X3) / 3)
                
                # Pindahkan mata kuliah ke posisi baru jika memungkinkan
                new_schedule = move_course_to_new_position(original_schedule, slot_id, new_position)
                if new_schedule:
                    new_fitness = calculate_fitness(new_schedule)
                    if new_fitness <= wolves[i]['fitness']:
                        wolves[i]['schedule'] = new_schedule
                        wolves[i]['fitness'] = new_fitness
                        improved = True
            
            # Jika tidak ada perbaikan, lakukan mutasi acak
            if not improved and random.random() < 0.5:
                mutated_schedule = mutate_schedule(original_schedule)
                new_fitness = calculate_fitness(mutated_schedule)
                if new_fitness <= wolves[i]['fitness']:
                    wolves[i]['schedule'] = mutated_schedule
                    wolves[i]['fitness'] = new_fitness
        
        # Urutkan wolves berdasarkan fitness
        wolves.sort(key=lambda x: x['fitness'])
        
        # Update alpha, beta, dan delta
        alpha = wolves[0]
        beta = wolves[1]
        delta = wolves[2]
        
        # Simpan history
        history.append({
            'iteration': iteration,
            'best_fitness': alpha['fitness'],
            'average_fitness': sum(wolf['fitness'] for wolf in wolves) / num_wolves
        })
        
        print(f"Iterasi {iteration+1}/{max_iterations}: Best Fitness = {alpha['fitness']}")
        
        # Kondisi berhenti jika sudah optimal
        if alpha['fitness'] == 0:
            print("Solusi optimal ditemukan!")
            break
    
    return alpha['schedule'], history

def get_conflict_slots(schedule):
    """
    Mengidentifikasi slot yang terlibat dalam konflik
    """
    conflict_slots = set()
    teacher_intervals = defaultdict(list)
    room_intervals = defaultdict(list)
    
    # Kumpulkan semua interval waktu untuk dosen dan ruangan
    for slot in schedule:
        if not slot['mata_kuliah']:
            continue
        
        # Konversi waktu
        start = time_to_minutes(slot['jam_mulai'])
        end = time_to_minutes(slot['jam_selesai'])
        
        # Catat interval dosen
        teacher_intervals[(slot['dosen'], slot['hari'])].append((start, end, slot['id_slot']))
        
        # Catat interval ruangan jika offline
        if slot['metode'] != 'Online':
            room_intervals[(slot['ruang'], slot['hari'])].append((start, end, slot['id_slot']))
        
        # Cek preferensi khusus
        if "Ardiansyah, Dr., S.T., M.Cs." in str(slot['dosen']) and start <= 720:
            conflict_slots.add(slot['id_slot'])
        
        if "Ali Tarmuji, S.T., M.Cs." in str(slot['dosen']) and slot['hari'] == 'sabtu':
            conflict_slots.add(slot['id_slot'])
    
    # Cek konflik dosen
    for key, intervals in teacher_intervals.items():
        intervals.sort()
        for i in range(1, len(intervals)):
            if intervals[i][0] < intervals[i-1][1]:
                conflict_slots.add(intervals[i-1][2])
                conflict_slots.add(intervals[i][2])
    
    # Cek konflik ruangan
    for key, intervals in room_intervals.items():
        intervals.sort()
        for i in range(1, len(intervals)):
            if intervals[i][0] < intervals[i-1][1]:
                conflict_slots.add(intervals[i-1][2])
                conflict_slots.add(intervals[i][2])
    
    return list(conflict_slots)

def find_slot_position(schedule, slot_id):
    """
    Menemukan posisi slot dalam jadwal, digunakan untuk perhitungan GWO
    """
    for i, slot in enumerate(schedule):
        if slot['id_slot'] == slot_id:
            return i
    return -1

def move_course_to_new_position(schedule, slot_id, new_position):
    """
    Memindahkan mata kuliah dari slot yang bermasalah ke posisi baru
    """
    schedule_copy = copy.deepcopy(schedule)
    
    # Temukan slot saat ini
    current_pos = find_slot_position(schedule_copy, slot_id)
    if current_pos == -1:
        return None
    
    # Pastikan posisi baru valid
    new_position = max(0, min(new_position, len(schedule_copy) - 1))
    
    # Jika new_position dan current_pos sama, coba slot terdekat
    if new_position == current_pos:
        alternatives = [new_position + 1, new_position - 1, new_position + 2, new_position - 2]
        alternatives = [pos for pos in alternatives if 0 <= pos < len(schedule_copy)]
        if alternatives:
            new_position = random.choice(alternatives)
    
    # Ambil informasi mata kuliah dari slot saat ini
    current_slot = schedule_copy[current_pos]
    if not current_slot['mata_kuliah']:
        return schedule_copy
    
    target_slot = schedule_copy[new_position]
    
    # Cek apakah target slot tersedia
    if target_slot['mata_kuliah'] is not None:
        # Jika target sibuk, cari slot kosong terdekat
        for offset in range(1, 10):  # Coba beberapa offset terdekat
            candidates = []
            if new_position + offset < len(schedule_copy) and schedule_copy[new_position + offset]['mata_kuliah'] is None:
                candidates.append(new_position + offset)
            if new_position - offset >= 0 and schedule_copy[new_position - offset]['mata_kuliah'] is None:
                candidates.append(new_position - offset)
            
            if candidates:
                new_position = random.choice(candidates)
                target_slot = schedule_copy[new_position]
                break
        else:
            # Jika tidak ada slot kosong terdekat, cari slot acak
            empty_slots = [i for i, s in enumerate(schedule_copy) if s['mata_kuliah'] is None]
            if empty_slots:
                new_position = random.choice(empty_slots)
                target_slot = schedule_copy[new_position]
            else:
                # Jika tidak ada slot kosong, tukar dengan slot acak
                random_pos = random.randint(0, len(schedule_copy) - 1)
                while random_pos == current_pos:
                    random_pos = random.randint(0, len(schedule_copy) - 1)
                new_position = random_pos
                target_slot = schedule_copy[new_position]
    
    # Pindahkan informasi antar slot
    course_info = {
        'mata_kuliah': current_slot['mata_kuliah'],
        'dosen': current_slot['dosen'],
        'kelas': current_slot['kelas'],
        'sks': current_slot['sks'],
        'metode': current_slot['metode']
    }
    
    # Kosongkan slot saat ini
    current_slot.update({
        'mata_kuliah': None,
        'dosen': None,
        'kelas': None,
        'sks': None,
        'metode': None
    })
    
    # Isi slot target
    target_slot.update(course_info)
    # Jika metode online, update nilai ruang
    if course_info['metode'] == 'Online':
        target_slot['ruang'] = 'Online'
    
    return schedule_copy

def mutate_schedule(schedule):
    """
    Melakukan mutasi acak pada jadwal untuk eksplorasi
    """
    schedule_copy = copy.deepcopy(schedule)
    
    # Pilih dua slot acak yang berisi mata kuliah
    filled_slots = [i for i, slot in enumerate(schedule_copy) if slot['mata_kuliah'] is not None]
    
    if len(filled_slots) < 2:
        return schedule_copy
    
    idx1, idx2 = random.sample(filled_slots, 2)
    
    # Tukar informasi mata kuliah antara dua slot
    slot1 = schedule_copy[idx1]
    slot2 = schedule_copy[idx2]
    
    # Simpan informasi slot1
    temp_info = {
        'mata_kuliah': slot1['mata_kuliah'],
        'dosen': slot1['dosen'],
        'kelas': slot1['kelas'],
        'sks': slot1['sks'],
        'metode': slot1['metode']
    }
    
    # Pindahkan informasi dari slot2 ke slot1
    slot1.update({
        'mata_kuliah': slot2['mata_kuliah'],
        'dosen': slot2['dosen'],
        'kelas': slot2['kelas'],
        'sks': slot2['sks'],
        'metode': slot2['metode']
    })
    
    # Jika metode online, update nilai ruang
    if slot2['metode'] == 'Online':
        slot1['ruang'] = 'Online'
    
    # Pindahkan informasi dari slot1 ke slot2
    slot2.update(temp_info)
    
    # Jika metode online, update nilai ruang
    if temp_info['metode'] == 'Online':
        slot2['ruang'] = 'Online'
    
    return schedule_copy

def visualize_schedule(schedule, title="Jadwal Kuliah"):
    """
    Memvisualisasikan jadwal dalam bentuk tabel
    """
    import matplotlib.pyplot as plt
    import pandas as pd
    
    # Filter hanya slot yang terisi
    filled_slots = [slot for slot in schedule if slot['mata_kuliah'] is not None]
    
    # Buat DataFrame untuk visualisasi
    schedule_df = pd.DataFrame([{
        'Mata Kuliah': slot['mata_kuliah'],
        'Dosen': slot['dosen'],
        'Kelas': slot['kelas'],
        'Ruang': slot['ruang'],
        'Hari': slot['hari'],
        'Waktu': f"{slot['jam_mulai']} - {slot['jam_selesai']}",
        'Metode': slot['metode']
    } for slot in filled_slots])
    
    # Urutkan berdasarkan hari dan waktu
    hari_order = {'senin': 0, 'selasa': 1, 'rabu': 2, 'kamis': 3, 'jumat': 4, 'sabtu': 5, 'minggu': 6}
    schedule_df['hari_order'] = schedule_df['Hari'].map(lambda x: hari_order.get(x.lower(), 7))
    schedule_df = schedule_df.sort_values(['hari_order', 'Waktu'])
    schedule_df = schedule_df.drop('hari_order', axis=1)
    
    return schedule_df

def run_gwo_scheduling(merged_df, max_iterations=50, num_wolves=10):
    """
    Menjalankan algoritma GWO dan mengembalikan jadwal terbaik
    """
    # Jalankan algoritma GWO
    best_schedule, history = grey_wolf_optimizer(
        merged_df=merged_df,
        max_iterations=max_iterations,
        num_wolves=num_wolves,
        a_start=2.0,
        a_end=0.0
    )
    
    # Visualisasikan hasil
    schedule_df = visualize_schedule(best_schedule, "Jadwal Optimized dengan GWO")
    
    # Cetak statistik
    final_fitness = calculate_fitness(best_schedule)
    print(f"Fitness akhir: {final_fitness}")
    
    return best_schedule, schedule_df, history

# Jalankan algoritma GWO
best_schedule, schedule_df, history = run_gwo_scheduling(
    merged_df=merged_df,
    max_iterations=50,  # Dapat disesuaikan
    num_wolves=10       # Dapat disesuaikan
)

# Tampilkan jadwal dalam bentuk DataFrame
print(schedule_df)

# Cek nilai fitness akhir (penalty)
print(f"Penalty akhir: {calculate_fitness(best_schedule)}")

Iterasi 1/50: Best Fitness = 9.5
Iterasi 2/50: Best Fitness = 9.5
Iterasi 3/50: Best Fitness = 9.0
Iterasi 4/50: Best Fitness = 8.5
Iterasi 5/50: Best Fitness = 8.0
Iterasi 6/50: Best Fitness = 7.5
Iterasi 7/50: Best Fitness = 7.0
Iterasi 8/50: Best Fitness = 6.5
Iterasi 9/50: Best Fitness = 6.0
Iterasi 10/50: Best Fitness = 5.0
Iterasi 11/50: Best Fitness = 4.5
Iterasi 12/50: Best Fitness = 4.0
Iterasi 13/50: Best Fitness = 3.5
Iterasi 14/50: Best Fitness = 3.0
Iterasi 15/50: Best Fitness = 2.0
Iterasi 16/50: Best Fitness = 1.5
Iterasi 17/50: Best Fitness = 1.5
Iterasi 18/50: Best Fitness = 1.5
Iterasi 19/50: Best Fitness = 1.0
Iterasi 20/50: Best Fitness = 1.0
Iterasi 21/50: Best Fitness = 0.5
Iterasi 22/50: Best Fitness = 0.5
Iterasi 23/50: Best Fitness = 0.5
Iterasi 24/50: Best Fitness = 0.5
Iterasi 25/50: Best Fitness = 0
Solusi optimal ditemukan!
Fitness akhir: 0
                                         Mata Kuliah  \
3                            Pemrosesan Bahasa Alami   
13    